# With statement in Python

In [ ]:
# 'With' is good for (i) error/exception handling, (ii) memory usage (open/close files), (iii) readability!
with open('./Week_2/example.txt', 'r') as f:
    for a_line in f:
        pass # do something

# Another way:
f = open('./Week_2/example.txt', 'r')
for a_line in f:
    pass # do something
f.close()

# This might be not bad because Python automatically closes and empties memory after running the for loop,
# But the 'with' statement is a legitimate way
for a_line in open('./Week_2/example.txt', 'r'):
    pass # do something

# AddHs

In [ ]:
from rdkit import Chem
mol = Chem.MolFromSmiles('CCO')
print([atom.GetSymbol() for atom in mol.GetAtoms()]  )
mol = Chem.AddHs(mol)
print([atom.GetSymbol() for atom in mol.GetAtoms()]  )

# List vs. Tuple

In [ ]:
a = [1,2,3]
a[1] = 4
print(a)

b = (1,2,3)
b[1] = 4
# 'tuple' object does not support item assignment

In [ ]:
#List of tuples

list_of_tuples= []
for i in range(4):
    for j in range(i+1, 4):
        list_of_tuples.append((i,j))
print(list_of_tuples)

list_of_lists = []
for i in range(4):
    for j in range(i+1, 4):
        list_of_lists.append([i,j])
print(list_of_lists)

# Tuple is good for (i) managing an array of 'constants', \
#(ii) memory usage (generally less memory consumption than list), (iii) readability!

# GetBonds

In [ ]:
#The atom indices of each bond do not change, so here I use tuple for saving atom indices
mol = Chem.MolFromSmiles('CCO')
mol = Chem.AddHs(mol)

bonds = []
for bond in mol.GetBonds():
    a1_idx = bond.GetBeginAtom().GetIdx()
    a2_idx = bond.GetEndAtom().GetIdx()
    
    bonds.append((a1_idx, a2_idx))
    
    atom1, atom2 = mol.GetAtomWithIdx(a1_idx), mol.GetAtomWithIdx(a2_idx)
    
    print(atom1.GetSymbol(), atom2.GetSymbol())
    
print(bonds)

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True
mol

# Dictionary

In [ ]:
# Example 1
from rdkit.Chem.Descriptors import MolWt, NumRadicalElectrons
from rdkit.Chem.rdmolops import GetFormalCharge
# integer, float, string can be dictionary 'keys' which are used for indexing
mol_properties = {}
mol_properties['MolWt'] = MolWt(mol)
mol_properties['GetFormalCharge'] = GetFormalCharge(mol)
mol_properties['AtomSymbols'] = [atom.GetSymbol() for atom in mol.GetAtoms()]

print(mol_properties)
print(mol_properties['MolWt'])

In [ ]:
#dictionary keys
for key in mol_properties.keys():
    print(key, mol_properties[key])

In [ ]:
#Example 2
atom_dict = {}

atom_dict['C'] = []
atom_dict['H'] = []
atom_dict['O'] = []
for i, atom in enumerate(mol.GetAtoms()):
    print(i, atom.GetSymbol())
    atom_dict[atom.GetSymbol()].append(i)
    
print(atom_dict)
    
## --- OR ---
atom_dict2 = {}
mol_all_elements = list(set(atom.GetSymbol() for atom in mol.GetAtoms()))

for element in mol_all_elements:
    atom_dict2[element] = [i for i, atom in enumerate(mol.GetAtoms()) \
                          if atom.GetSymbol() == element]
print(atom_dict2)

## --- OR ---
atom_dict3 = {}
for i, atom in enumerate(mol.GetAtoms()):
    #atom_dict3[atom.GetSymbol()].append(i)
    try:
        atom_dict3[atom.GetSymbol()].append(i)
    except KeyError:
        atom_dict3[atom.GetSymbol()] = [i]

print(atom_dict3)

# Please let me know after the meeting if you know any better ways to do this

# Exercise 1

Make a dictionary of the bonds of the molecule 'CCO'

Keys: bond types (C-C, C-H, etc.), each key has a list of tuples of atom indices.

Bond types should be in an alphabetical order (e.g. H-C --> C-H)

# PubChemPy

In the data experimentalists give us or in experimental databases, molecules are usually described as IUPAC name or CAS number rather than SMILES. So, I wanted to cover PubChemPy which is one of the ways to convert IUPAC name or CAS number into SMILES. Also, molecular properties can be obtained in dictionary format.

In [ ]:
import pubchempy as pcp

# Since pcp.get_compounds returns a list of compounds (usually one compound, but sometimes >1), so index 0 is called
glucose = pcp.get_compounds('glucose','name')[0]
caffeine = pcp.get_compounds('58-08-2','name')[0]

caffeine_dict = caffeine.to_dict()
print(caffeine_dict.keys())
print('-------------------')
print(caffeine_dict['canonical_smiles'])
print(caffeine.canonical_smiles)
print('-------------------')

print(caffeine.to_dict(properties = ['canonical_smiles', 'tpsa','atoms', 'bonds']))

# Read and write json file

In [ ]:
import json
#write json
with open('./Week_3/caffeine.json', 'w') as f:
    json.dump(caffeine_dict, f, indent = 4)

#read json
with open('./Week_3/caffeine.json','r') as f:
    data = json.load(f)
    for key in data.keys():
        print(key, data[key])

# Exercise 2

(1) open ./Week_3/smi_list_from_Week_2, parse a list of smiles 

(2) make dictionary, classify them in terms of the total number of atoms

The dictionary should look like:

{ 5: ['O=CO', 'C'], 10: ['CCO[C-]=O', 'C=C=CC', 'C=COC', 'CCOO', 'COOC', 'C1COC1', 'CC1CO1', 'CC#CC', 'O=C1C=CCO1', 'CC1=CC1', 'c1ccccc#1', 'O=C1OCCO1', 'C=C1CC1', 'C#CC(=O)OC', 'C#CC#CC#CC#C', 'CC(C)=O', 'C#CCC', 'OCCO', 'CCC=O', 'O=C1CC(=O)C1'], ... }

(3) Write ./Week_3/exercise_2.json .

# Assignment - Week 3

(1) open ./Week_3/smi_list_from_Week_2, parse a list of smiles 


(2) open ./Week_3/Week_3_iupac_name_cas_no, convert iupac or cas number to a smiles string

If pubchempy returns multiple compounds, please use the 0th one. 

If pubchempy returns no compounds, please just skip it.


(3) Merge (1) and (2), remove duplicates

Please note that the canonical smiles from PubChem needs to be 're-canonicalized' in RDKit to remove duplicates


(4) Collect the molecules that consist of C or H or O or N (and no other elements) and not more than 20 nonhydrogen atoms (mol.GetNumHeavyAtoms()), classify them in terms of stoichiometry

The dictionary should look like:

{'C11H22O2': ['CCCCCCC(=O)OC(C)CC', 'CCCCCCC1COC(C)(C)O1', 'CCCCC(CCCC)C(=O)OC', 'CCCCC(CC)COC(=O)CC', 'CCCCCCCCCOC(C)=O', 'CCCCCCOC(=O)CCCC', 'CCCCCOC(=O)CCCCC', 'CCCCCCCOC(=O)CCC', 'CCCCC(=O)OC(C)CCCC', 'CCCCCCCC(=O)OCCC', 'CCCCCCC(C)(C)CC(=O)O', 'CCCCCCC(=O)OCCCC', 'CCCCCCC1(C)OCCCO1', 'CCCCCCCC(=O)OC(C)C', 'CC(C)CCCC(C)CCC(=O)O', 'CCCCCCC(=O)OC(C)(C)C', 'CCCCCCCCOC(=O)CC', 'CCCCCC(C)OC(=O)CCC', 'CCCCCCCCCCC(=O)O', 'CCCCCCCCC(C)C(=O)O', 'CCCCCCCCCCOC=O', 'CCCCCCCCC(=O)OCC', 'CCOC(=O)CCCCCC(C)C', 'CCOC(=O)CCCCC(C)CC', 'CCCCCCCCCC(=O)OC', 'CCCCCCCOC(=O)C(C)C'], 'H2O': ['O'], ... }

Avoid writing '1's. e.g.) H2O1 --> H2O

Follow alphabetical orders. e.g.) 'C8H10N4O2'

(5) Write ./Week_3/Assignment.json .


In [ ]:
# If pubchempy returns no compounds, please just skip it. Perhaps try-except can be used?
# Maybe many molecules are skipped. I brought the molecules quite randomly including very tricky ones, 
# and PubChemPy is not always perfect.
a = []
print(a[0]) #IndexError

'''
try:
   # do something
except IndexError:
    continue # skip this cycle and proceed to the next cycle
'''